# Set up

In [ ]:
!gdown --id 1Q3OexUDv1-U06DpLZNKZLNwFCGCIw5CK

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Q3OexUDv1-U06DpLZNKZLNwFCGCIw5CK
To: /content/thsarabunnew-webfont.ttf
100% 98.3k/98.3k [00:00<00:00, 52.7MB/s]


In [ ]:
!pip install deepcut

     |████████████████████████████████| 2.0 MB 5.1 MB/s 


In [ ]:
!pip -q install python-crfsuite pythainlp[full]
!pip install -q gensim datasets transformers[sentencepiece]

     |████████████████████████████████| 965 kB 4.9 MB/s 
     |████████████████████████████████| 11.5 MB 37.9 MB/s 
     |████████████████████████████████| 1.2 MB 46.0 MB/s 
     |████████████████████████████████| 1.7 MB 31.7 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 1.5 MB 46.0 MB/s 
     |████████████████████████████████| 1.5 MB 28.7 MB/s 
     |████████████████████████████████| 1.5 MB 43.2 MB/s 
     |████████████████████████████████| 1.3 MB 40.6 MB/s 
     |████████████████████████████████| 8.7 MB 30.8 MB/s 
     |████████████████████████████████| 4.0 MB 45.0 MB/s 
     |████████████████████████████████| 44.2 MB 81 kB/s 
     |████████████████████████████████| 24.1 MB 12.8 MB/s 
     |████████████████████████████████| 18.2 MB 425 kB/s 
     |████████████████████████████████| 305 kB 62.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
    

In [ ]:
#standard
import re
import pandas as pd
import numpy as np
import pickle
import string
from tqdm import tqdm

#python nlp (Thai)
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize, subword_tokenize

#tranformers
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, pipeline

# For visualization
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
# params
pd.set_option("max_colwidth", 0)
sns.set_context("talk")
sns.set_palette(sns.color_palette("colorblind", 10))
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf') 
mpl.rc('font', family='TH Sarabun New')

%matplotlib inline

In [ ]:
#load dataset
thai_toxic = load_dataset('thai_toxicity_tweet')
thai_toxic = pd.DataFrame(thai_toxic['train'])
thai_toxic.head()

Reusing dataset thai_toxicity_tweet (/root/.cache/huggingface/datasets/thai_toxicity_tweet/thai_toxicity_tweet/1.0.0/6156251739b83a8e57f9ed4adf0f1d134e9bfded6d2870647558fdb7a0e60306)


  0%|          | 0/1 [00:00<?, ?it/s]

,tweet_id,tweet_text,toxic_votes,nontoxic_votes,is_toxic
0,898576382384418817,วันๆ นี่คุยกะหมา แมว หมู ไก่ ม้า ควาย มากกว่าคุยกับคนไปละ,0,3,0
1,847331851425300481,TWEET_NOT_FOUND,3,0,1
2,899587505493692417,หล่อมากกก หล่อวัวตายควายล้มกันเลยทีเดียวววว,0,3,0
3,898920493763280897,สิวเหี้ยไรขึ้นหลังหู เสียชาติเกิดมั้ย เกิดมามันต้องต้องโดดเด่นเด้งดึ๋งอย่าเช่นขึ้นที่หน้าไรงี้ อีควาย,0,3,0
4,896808964326694912,อ่ะ ป่วยก็ป่วย งานจ้างก็ต้องทำ งานเรียนก็ต้องทำ วิชาดีไซน์ตัวร้ายกับควายอย่างกูเอง,0,3,0


# Tokenization

In [ ]:
short_text = "ถอยหมอน"
text = "อะจ๊ะเอ๋ตัวเอง ท่านผู้เจริญผู้ซึ่งมากไปด้วยปัญญา อะฮ่า"
long_text = "สวัสดี ฉันชื่อ คิระ โยชิคาเงะ อายุ 33 ปี อาศัยอยู่ย่านที่พักอาศัยทางตะวันออกเฉียงเหนือของเมืองโมริโอ ยังโสด ทำงานที่ห้างคาเมยู ทุกวันจะกลับบ้านก่อนสองทุ่ม ไม่สูบบุหรี่แต่ดื่มไวน์นิดหน่อย เข้านอนก่อนห้าทุ่มฉันต้องนอนวันละแปดชั่วโมง ก่อนนอนจะดื่มนมอุ่นๆหนึ่งแก้วและยืดเส้นยืดสาย20นาทีเพื่อผ่อนคลายร่างกายก่อนเข้านอนจะได้หลับสนิทตลอดทั้งคืน ฉันตื่นนอนเหมือนเด็กทารกปราศจากความเหนื่อยล้าและความเครียด ผลตรวจร่างกายก็ปกติ ฉันแค่จะบอกว่าฉันอยากอยู่อย่างสงบ ไม่ต้องการแข่งขันกับใครและไม่อยากสร้างปัญหาให้ตัวเองปวดหัวหรือสร้างศัตรูจนทำให้นอนไม่หลับ นั่นคือทัศนคติที่ฉันมีต่อสังคมและฉันรู้ว่ามันจะทำให้ฉันมีความสุข"

## sentence level

In [ ]:
from pythainlp.tokenize import sent_tokenize

In [ ]:
# crfcut engine (split by CRF trained on TED dataset)
%%time
print(sent_tokenize(text))
print('\n'.join(sent_tokenize(long_text)))

['อะจ๊ะเอ๋ตัวเอง ', 'ท่านผู้เจริญผู้ซึ่งมากไปด้วยปัญญา ', 'อะฮ่า']
สวัสดี ฉันชื่อ คิระ โยชิคาเงะ 
อายุ 33 ปี อาศัยอยู่ย่านที่พักอาศัยทางตะวันออกเฉียงเหนือของเมืองโมริโอ 
ยังโสด ทำงานที่ห้างคาเมยู ทุกวันจะกลับบ้านก่อนสองทุ่ม 
ไม่สูบบุหรี่แต่ดื่มไวน์นิดหน่อย เข้านอนก่อนห้าทุ่มฉันต้องนอนวันละแปดชั่วโมง ก่อนนอนจะดื่มนมอุ่นๆหนึ่งแก้วและยืดเส้นยืดสาย20นาทีเพื่อผ่อนคลายร่างกายก่อนเข้านอนจะได้หลับสนิทตลอดทั้งคืน 
ฉันตื่นนอนเหมือนเด็กทารกปราศจากความเหนื่อยล้าและความเครียด 
ผลตรวจร่างกายก็ปกติ 
ฉันแค่จะบอกว่าฉันอยากอยู่อย่างสงบ 
ไม่ต้องการแข่งขันกับใครและไม่อยากสร้างปัญหาให้ตัวเองปวดหัวหรือสร้างศัตรูจนทำให้นอนไม่หลับ 
นั่นคือทัศนคติที่ฉันมีต่อสังคมและฉันรู้ว่ามันจะทำให้ฉันมีความสุข
CPU times: user 12.3 ms, sys: 1.75 ms, total: 14.1 ms
Wall time: 16.9 ms


In [ ]:
# whitespace engine
%%time
print(sent_tokenize(text, engine="whitespace"))
print('\n'.join(sent_tokenize(long_text, engine="whitespace")))

['อะจ๊ะเอ๋ตัวเอง', 'ท่านผู้เจริญผู้ซึ่งมากไปด้วยปัญญา', 'อะฮ่า']
สวัสดี
ฉันชื่อ
คิระ
โยชิคาเงะ
อายุ
33
ปี
อาศัยอยู่ย่านที่พักอาศัยทางตะวันออกเฉียงเหนือของเมืองโมริโอ
ยังโสด
ทำงานที่ห้างคาเมยู
ทุกวันจะกลับบ้านก่อนสองทุ่ม
ไม่สูบบุหรี่แต่ดื่มไวน์นิดหน่อย
เข้านอนก่อนห้าทุ่มฉันต้องนอนวันละแปดชั่วโมง
ก่อนนอนจะดื่มนมอุ่นๆหนึ่งแก้วและยืดเส้นยืดสาย20นาทีเพื่อผ่อนคลายร่างกายก่อนเข้านอนจะได้หลับสนิทตลอดทั้งคืน
ฉันตื่นนอนเหมือนเด็กทารกปราศจากความเหนื่อยล้าและความเครียด
ผลตรวจร่างกายก็ปกติ
ฉันแค่จะบอกว่าฉันอยากอยู่อย่างสงบ
ไม่ต้องการแข่งขันกับใครและไม่อยากสร้างปัญหาให้ตัวเองปวดหัวหรือสร้างศัตรูจนทำให้นอนไม่หลับ
นั่นคือทัศนคติที่ฉันมีต่อสังคมและฉันรู้ว่ามันจะทำให้ฉันมีความสุข
CPU times: user 1.57 ms, sys: 999 µs, total: 2.57 ms
Wall time: 2.91 ms


In [ ]:
# whitespace+newline engine
%%time
print(sent_tokenize(text, engine="whitespace+newline"))
print('\n'.join(sent_tokenize(long_text, engine="whitespace+newline")))

['อะจ๊ะเอ๋ตัวเอง', 'ท่านผู้เจริญผู้ซึ่งมากไปด้วยปัญญา', 'อะฮ่า']
สวัสดี
ฉันชื่อ
คิระ
โยชิคาเงะ
อายุ
33
ปี
อาศัยอยู่ย่านที่พักอาศัยทางตะวันออกเฉียงเหนือของเมืองโมริโอ
ยังโสด
ทำงานที่ห้างคาเมยู
ทุกวันจะกลับบ้านก่อนสองทุ่ม
ไม่สูบบุหรี่แต่ดื่มไวน์นิดหน่อย
เข้านอนก่อนห้าทุ่มฉันต้องนอนวันละแปดชั่วโมง
ก่อนนอนจะดื่มนมอุ่นๆหนึ่งแก้วและยืดเส้นยืดสาย20นาทีเพื่อผ่อนคลายร่างกายก่อนเข้านอนจะได้หลับสนิทตลอดทั้งคืน
ฉันตื่นนอนเหมือนเด็กทารกปราศจากความเหนื่อยล้าและความเครียด
ผลตรวจร่างกายก็ปกติ
ฉันแค่จะบอกว่าฉันอยากอยู่อย่างสงบ
ไม่ต้องการแข่งขันกับใครและไม่อยากสร้างปัญหาให้ตัวเองปวดหัวหรือสร้างศัตรูจนทำให้นอนไม่หลับ
นั่นคือทัศนคติที่ฉันมีต่อสังคมและฉันรู้ว่ามันจะทำให้ฉันมีความสุข
CPU times: user 593 µs, sys: 43 µs, total: 636 µs
Wall time: 536 µs


In [ ]:
# tltk engine
%%time
print(sent_tokenize(text, engine="tltk"))
print('\n'.join(sent_tokenize(long_text, engine="tltk")))

['อะจ๊ะเอ๋ตัวเอง ท่านผู้เจริญผู้ซึ่งมากไปด้วยปัญญา อะฮ่า']
สวัสดี ฉันชื่อ คิระ โยชิคาเงะ อายุ 33 ปี อาศัยอยู่ย่านที่พักอาศัยทางตะวันออกเฉียงเหนือของเมืองโมริโอ ยังโสด ทำงานที่ห้างคาเมยู ทุกวันจะกลับบ้านก่อนสองทุ่ม ไม่สูบบุหรี่แต่ดื่มไวน์นิดหน่อย เข้านอนก่อนห้าทุ่มฉันต้องนอนวันละแปดชั่วโมง ก่อนนอนจะดื่มนมอุ่นๆ
หนึ่งแก้วและยืดเส้นยืดสาย20นาทีเพื่อผ่อนคลายร่างกายก่อนเข้านอนจะได้หลับสนิทตลอดทั้งคืน ฉันตื่นนอนเหมือนเด็กทารกปราศจากความเหนื่อยล้าและความเครียด ผลตรวจร่างกายก็ปกติ ฉันแค่จะบอกว่าฉันอยากอยู่อย่างสงบ ไม่ต้องการแข่งขันกับใครและไม่อยากสร้างปัญหาให้ตัวเองปวดหัวหรือสร้างศัตรูจนทำให้นอนไม่หลับ นั่นคือทัศนคติที่ฉันมีต่อสังคมและฉันรู้ว่ามันจะทำให้ฉันมีความสุข
CPU times: user 5.63 s, sys: 12.9 ms, total: 5.64 s
Wall time: 5.63 s


## clause level

need list of word for input

In [ ]:
from pythainlp.tokenize import clause_tokenize

In [ ]:
%%time
print(clause_tokenize(['ถอย','หมอน']))
print(clause_tokenize(['อะ','จ๊ะเอ๋','ตัวเอง','ท่าน','ผู้เจริญ','ผู้','ซึ่ง','มาก','ไป','ด้วย','ปัญญา','อะฮ่า']))

[['ถอย', 'หมอน']]
[['อะ', 'จ๊ะเอ๋', 'ตัวเอง', 'ท่าน', 'ผู้เจริญ', 'ผู้'], ['ซึ่ง', 'มาก', 'ไป', 'ด้วย', 'ปัญญา', 'อะฮ่า']]
CPU times: user 2.9 ms, sys: 1.02 ms, total: 3.92 ms
Wall time: 3.82 ms


## word level

### dictionary base

In [ ]:
# longest maching (ตัดคำให้ยาวที่สุด)
%%time
print('|'.join(word_tokenize(short_text, engine='longest')))
print('|'.join(word_tokenize(text, engine='longest')))

ถอย|หมอน
อะ|จ๊ะเอ๋|ตัวเอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 2.17 ms, sys: 0 ns, total: 2.17 ms
Wall time: 2.89 ms


In [ ]:
# maximal maching + Thai Character Cluster
# (ตัดคำให้มีจน.น้อยที่สุด และจน.คำที่ไม่มีความหมายน้อยที่สุด)
%%time
print('|'.join(word_tokenize(short_text, engine='newmm')))
print('|'.join(word_tokenize(text, engine='newmm')))

ถอย|หมอน
อะ|จ๊ะเอ๋|ตัวเอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 2.44 ms, sys: 0 ns, total: 2.44 ms
Wall time: 7.91 ms


In [ ]:
# maximal maching + Thai Character Cluster (TCC) boundaries
%%time
print('|'.join(word_tokenize(short_text, engine='nercut')))
print('|'.join(word_tokenize(text, engine='nercut')))

Corpus: thainer
- Downloading: thainer 1.5


100%|██████████| 1637304/1637304 [00:00<00:00, 29065567.57it/s]

ถอย|หมอน
อะ|จ๊ะเอ๋|ตัวเอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 85.3 ms, sys: 8.97 ms, total: 94.3 ms
Wall time: 662 ms


In [ ]:
# maximal maching + Thai Character Cluster boundaries for rust/node.js
%%time
print('|'.join(word_tokenize(short_text, engine='nlpo3')))
print('|'.join(word_tokenize(text, engine='nlpo3')))

ถอย|หมอน
อะ|จ๊ะเอ๋|ตัวเอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 102 ms, sys: 51.5 ms, total: 153 ms
Wall time: 161 ms


In [ ]:
# maximal maching
%%time
print('|'.join(word_tokenize(short_text, engine='multi_cut')))
print('|'.join(word_tokenize(text, engine='multi_cut')))

ถอย|หมอน
อะ|จ๊ะเอ๋|ตัวเอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 10.3 ms, sys: 912 µs, total: 11.2 ms
Wall time: 13 ms


### machine learning base

In [ ]:
# sefr cut (Stacked Ensemble Filter and Refine for Word Segmentation)
%%time
print('|'.join(word_tokenize(short_text, engine='sefr_cut')))
print('|'.join(word_tokenize(text, engine='sefr_cut')))

loading model.....
Success
ถอย|หมอน
อะ|จ๊ะ|เอ๋|ตัว|เอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 1.5 s, sys: 95.8 ms, total: 1.6 s
Wall time: 2.24 s


### deep learning base

In [ ]:
# AttaCut (CNN base)
%%time
print('|'.join(word_tokenize(short_text, engine='attacut')))
print('|'.join(word_tokenize(text, engine='attacut')))

ถอย|หมอน
อะจ๊ะเอ๋|ตัว|เอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะฮ่า
CPU times: user 65.2 ms, sys: 3.65 ms, total: 68.8 ms
Wall time: 128 ms


In [ ]:
# DeepCut (CNN base)
%%time
print('|'.join(word_tokenize(short_text, engine='deepcut')))
print('|'.join(word_tokenize(text, engine='deepcut')))

ถอยหมอน
อะจ๊ะ|เอ๋|ตัว|เอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญญา อะฮ่า
CPU times: user 1.61 s, sys: 21.4 ms, total: 1.63 s
Wall time: 2.1 s


In [ ]:
# OSKut (Out-of-domain StacKed cut for Word Segmentation)
# CNN + Bidirectional LSTM + attention
%%time
print('|'.join(word_tokenize(short_text, engine='oskut')))
print('|'.join(word_tokenize(text, engine='oskut')))

ถอยหมอน
อะ|จ๊ะ|เอ๋|ตัว|เอง| |ท่าน|ผู้|เจริญ|ผู้ซึ่ง|มาก|ไป|ด้วย|ปัญญา| |อะ|ฮ่า
CPU times: user 420 ms, sys: 16.7 ms, total: 437 ms
Wall time: 530 ms


## subword level

In [ ]:
# tcc (Thai Character Clusters)
%%time
print('|'.join(subword_tokenize(short_text, engine='tcc')))
print('|'.join(subword_tokenize(text, engine='tcc')))

ถ|อ|ย|ห|ม|อ|น
อะ|จ๊ะ|เอ๋|ตัว|เอ|ง| |ท่า|น|ผู้|เจ|ริ|ญ|ผู้|ซึ่|ง|มา|ก|ไป|ด้|ว|ย|ปัญ|ญา| |อะ|ฮ่า
CPU times: user 2.37 ms, sys: 0 ns, total: 2.37 ms
Wall time: 2.39 ms


In [ ]:
# etcc (enhanced Thai Character Cluster)
%%time
print('|'.join(subword_tokenize(short_text, engine='etcc')))
print('|'.join(subword_tokenize(text, engine='etcc')))

ถ|อ|ย|ห|ม|อ|น
อะ|จ๊ะ|เอ๋ตัวเอ|ง| |ท่า|น|ผู้เจ|ริญ|ผู้|ซึ่|ง|มา|ก|ไป|ด้|ว|ย|ปัญ|ญา| |อะ|ฮ่า
CPU times: user 563 ms, sys: 75.5 ms, total: 639 ms
Wall time: 654 ms


In [ ]:
# dict (newmm word tokenizer with a syllable dictionary)
%%time
print('|'.join(subword_tokenize(short_text, engine='dict')))
print('|'.join(subword_tokenize(text, engine='dict')))

ถอย|หมอน
อะ|จ๊ะ|เอ๋|ตัว|เอง| |ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญ|ญา| |อะ|ฮ่า
CPU times: user 5.58 ms, sys: 0 ns, total: 5.58 ms
Wall time: 5.59 ms


In [ ]:
# ssg (CRF syllable segmenter for Thai)
%%time
print('|'.join(subword_tokenize(short_text, engine='ssg')))
print('|'.join(subword_tokenize(text, engine='ssg')))

ถอย|หมอน
อะ|จ๊ะ|เอ๋|ตัว|เอง| ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญ|ญา| |อะ|ฮ่า
CPU times: user 4.43 ms, sys: 15 µs, total: 4.45 ms
Wall time: 7.59 ms


In [ ]:
# tltk (syllable tokenizer from tltk)
%%time
print('|'.join(subword_tokenize(short_text, engine='tltk')))
print('|'.join(subword_tokenize(text, engine='tltk')))

ถอย|หมอน
อะ|จ๊ะ|เอ๋|ตัว|เอง|<s/>ท่าน|ผู้|เจริญ|ผู้|ซึ่ง|มาก|ไป|ด้วย|ปัญ|ญา|<s/>อะ|ฮ่า
CPU times: user 798 ms, sys: 3.2 ms, total: 802 ms
Wall time: 807 ms


In [ ]:
'''
%%time
print('|'.join(subword_tokenize(short_text, engine='wangchanberta')))
print('|'.join(subword_tokenize(text, engine='wangchanberta')))
'''

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399M [00:00<?, ?B/s]

▁|ถอย|หมอน
▁|อะ|จ๊ะ|เอ๋|ตัวเอง|▁ท่าน|ผู้|เจริญ|ผู้ซึ่ง|มาก|ไปด้วย|ปัญญา|▁|อะ|ฮ่า
CPU times: user 11.1 s, sys: 3.73 s, total: 14.8 s
Wall time: 18.1 s


In [ ]:
# wangchanberta (sentence pierce from bert model)
%%time
print('|'.join(subword_tokenize(short_text, engine='wangchanberta')))
print('|'.join(subword_tokenize(text, engine='wangchanberta')))

▁|ถอย|หมอน
▁|อะ|จ๊ะ|เอ๋|ตัวเอง|▁ท่าน|ผู้|เจริญ|ผู้ซึ่ง|มาก|ไปด้วย|ปัญญา|▁|อะ|ฮ่า
CPU times: user 2.82 ms, sys: 0 ns, total: 2.82 ms
Wall time: 7.94 ms


## syllable level

In [ ]:
%%time
print(list(short_text))
print(list(text))

['ถ', 'อ', 'ย', 'ห', 'ม', 'อ', 'น']
['อ', 'ะ', 'จ', '๊', 'ะ', 'เ', 'อ', '๋', 'ต', 'ั', 'ว', 'เ', 'อ', 'ง', ' ', 'ท', '่', 'า', 'น', 'ผ', 'ู', '้', 'เ', 'จ', 'ร', 'ิ', 'ญ', 'ผ', 'ู', '้', 'ซ', 'ึ', '่', 'ง', 'ม', 'า', 'ก', 'ไ', 'ป', 'ด', '้', 'ว', 'ย', 'ป', 'ั', 'ญ', 'ญ', 'า', ' ', 'อ', 'ะ', 'ฮ', '่', 'า']
CPU times: user 179 µs, sys: 12 µs, total: 191 µs
Wall time: 152 µs


syllable_tokenize in pythainlp now is deprecated (from developer) 